#  LU3MA201 : Projet / Travail d’étude et de recherche

<!-- dom:AUTHOR: Aya Bouzidi at [Sorbonne Université](http://www.sorbonne-universite.fr/), -->
<!-- Author: -->  
**Aya Bouzidi** ( L3 de Mathématiques à [Sorbonne Université](http://www.sorbonne-universite.fr/) ).

Licence <a href="https://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND</a>

# 3 Distance tangente comme algorithme de classification de chiffres manuscrits

<div id="ch:method_1"></div>

**Exemples de transformations importantes et leurs dérivées**

* **Translation d'axe (Ox)**: $T_X=\frac{\partial p}{\partial x}=p_x$

* **Translation d'axe (Oy)**: $T_Y=\frac{\partial p}{\partial y}=p_y$

* **Rotation de centre l'origine**: $T_X=y\frac{\partial p}{\partial x}-x\frac{\partial p}{\partial y}=yp_x-xp_y$

* **Scaling**: $T_X=x\frac{\partial p}{\partial x}+y\frac{\partial p}{\partial y}=xp_x+yp_y$

* **Transformation parallèle hyperbolique**: $T_X=x\frac{\partial p}{\partial x}-y\frac{\partial p}{\partial y}=xp_x-yp_y$

* **Transformation diagonale hyperbolique**: $T_X=y\frac{\partial p}{\partial x}+x\frac{\partial p}{\partial y}=yp_x+xp_y$

* **Thickening**: $T_X=(\frac{\partial p}{\partial x})^2+(\frac{\partial p}{\partial y})^2=(p_x)^2+(p_y)^2$

<div id="ch:method_1"></div>

Pour différentes transformations importantes du plan, notre algorithme est le suivant: 
    
* **Etape 1:** Pour chaque image de la base d'apprentissage et de la base de tests, calculer sa matrice tangente en fonction de la transformation choisie.


* **Etape 2:** Pour chaque image de la base de tests, calculer la distance tangente par rapport à toutes les images de la base d'apprentissage et la classifier selon le chiffre correspondant à l'image qui donne la plus petite distance tangente.

<div id="ch:method_1"></div>

**Les instructions suivantes permettent de charger les données de chiffres manuscrits disponibles dans les fichiers base_apprentissage.mat et base_test.mat :**

In [1]:
import scipy.io as spi
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
mat=spi.loadmat("base_apprentissage.mat")
data_train=np.transpose(mat['data'])
label_train=np.array(mat['label'])[0] #label: chiffre numérisé
label_train=label_train.astype(int) #Les labels sont stockés en flottants, on les convertit en entiers

mat = spi.loadmat("base_test.mat")
data_test = np.transpose(mat['data'])
label_test = np.array(mat['label'])[0]
label_test =label_test.astype(int)

In [3]:
# ajout overfit
data_test_overfit=data_train[:2000]

In [4]:
tuples=[]
for i in range(28):
    for j in range(28):
        tuples+=[(i,j)]
        
tuples_x=np.array([tuples[i][0] for i in range(784)])
tuples_y=np.array([tuples[i][1] for i in range(784)])

<div id="ch:method_1"></div>

**On lisse les images:**

In [5]:
def smooth(v):
    P=np.reshape(v,(28,28))
    def p(x,y):
        S=[P[i,j]*np.e**(-((x-i)**2+(y-j)**2)/(2*0.9**2)) for i,j in tuples]
        return sum(S)
    return p(tuples_x,tuples_y)

def smooth_fcl(v):
    P = np.reshape(v, (28,28))
    x, y = tuples_x, tuples_y
    return sum([P[i,j]*np.e**(-((x-i)**2+(y-j)**2)/(2*0.9**2)) for i,j in tuples])

In [6]:
smooth_train=[smooth_fcl(data_train[i]) for i in range(8000)]
smooth_test=[smooth_fcl(data_test[i]) for i in range(2000)]

In [7]:
#ajout overfit
label_test_overfit=label_train[:2000]
smooth_test_overfit=smooth_train[:2000]

<div id="ch:method_1"></div>

**On calcule les p_x pour les chiffres:**

In [8]:
def T_X(v):
    P=np.reshape(v,(28,28))
    P_=np.gradient(P)[0]
    return np.reshape(P_,(1,-1))[0]

In [9]:
p_x_train=[T_X(smooth_train[i]) for i in range(8000)]
p_x_test=[T_X(smooth_test[i]) for i in range(2000)]

In [10]:
#ajout overfit
p_x_test_overfit=[T_X(smooth_test_overfit[i]) for i in range(2000)]

<div id="ch:method_1"></div>

**On calcule les p_y pour les chiffres:**

In [11]:
def T_Y(v):
    P=np.reshape(v,(28,28))
    P_=np.gradient(P)[1]
    return np.reshape(P_,(1,-1))[0]

In [12]:
p_y_train=[T_Y(smooth_train[i]) for i in range(8000)]
p_y_test=[T_Y(smooth_test[i]) for i in range(2000)]

In [13]:
#ajout overfit
p_y_test_overfit=[T_Y(smooth_test_overfit[i]) for i in range(2000)]

## 3.1 Translation d'axe (Ox) 

<div id="ch:method_1"></div>

**Fonction de classification pour la X-translation:**

In [14]:
def estim_X(j): #estime l'image data_test[j]
    A=[np.hstack((np.reshape(-p_x_train[i],(-1,1)),np.reshape(p_x_test[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_test[j],(-1,1)) for i in range(8000)]
    résidus=[np.linalg.lstsq(A[i], b[i], rcond=None)[1][0] for i in range(8000)] 
    return label_train[résidus.index(min(résidus))]

In [15]:
# ajout overfit

def estim_X_overfit(j): #estime l'image data_test_overfit[j]
    A=[np.hstack((np.reshape(-p_x_train[i],(-1,1)),np.reshape(p_x_train[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_train[j],(-1,1)) for i in range(8000)]
    x=[np.linalg.lstsq(np.transpose(A[i])@A[i], np.transpose(A[i])@b[i],rcond=None)[0] for i in range(8000)]
    résidus=[np.linalg.norm(A[i]@x[i]-b[i]) for i in range(8000)]
    return label_train[résidus.index(min(résidus))]

In [16]:
# LISTES A STOCKER
%timeit
estim_x_trans=[estim_X(i) for i in range(2000)] # pour matrices de confusion
estim_x_trans_overfit=[estim_X_overfit(i) for i in range(2000)] # pour overfit

## 3.2 Translation d'axe (Oy) 

<div id="ch:method_1"></div>

**Fonction de classification pour la Y-translation:**

In [28]:
def estim_Y(j): #estime l'image data_test[j]
    A=[np.hstack((np.reshape(-p_y_train[i],(-1,1)),np.reshape(p_y_test[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_test[j],(-1,1)) for i in range(8000)]
    résidus=[np.linalg.lstsq(A[i], b[i], rcond=None)[1][0] for i in range(8000)] 
    return label_train[résidus.index(min(résidus))]

In [ ]:
# ajout overfit

def estim_Y_overfit(j): #estime l'image data_test_overfit[j]
    A=[np.hstack((np.reshape(-p_y_train[i],(-1,1)),np.reshape(p_y_train[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_train[j],(-1,1)) for i in range(8000)]
    x=[np.linalg.lstsq(np.transpose(A[i])@A[i], np.transpose(A[i])@b[i],rcond=None)[0] for i in range(8000)]
    résidus=[np.linalg.norm(A[i]@x[i]-b[i]) for i in range(8000)]
    return label_train[résidus.index(min(résidus))]

In [ ]:
# LISTES A STOCKER

estim_y_trans=[estim_Y(i) for i in range(2000)]
estim_y_trans_overfit=[estim_Y_overfit(i) for i in range(2000)]

## 3.3 Rotation

In [31]:
p_r_train=tuples_y*np.array(p_x_train)-tuples_x*np.array(p_y_train)
p_r_test=tuples_y*np.array(p_x_test)-tuples_x*np.array(p_y_test)

In [ ]:
# ajout overfit
p_r_test_overfit=tuples_y*np.array(p_x_test_overfit)-tuples_x*np.array(p_y_test_overfit)

<div id="ch:method_1"></div>

**Fonction de classification pour la rotation:**

In [33]:
def estim_R(j): #estime l'image data_test[j]
    A=[np.hstack((np.reshape(-p_r_train[i],(-1,1)),np.reshape(p_r_test[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_test[j],(-1,1)) for i in range(8000)]
    résidus=[np.linalg.lstsq(A[i], b[i], rcond=None)[1][0] for i in range(8000)] 
    return label_train[résidus.index(min(résidus))]

In [ ]:
# ajout overfit

def estim_R_overfit(j): #estime l'image data_test_overfit[j]
    A=[np.hstack((np.reshape(-p_r_train[i],(-1,1)),np.reshape(p_r_train[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_train[j],(-1,1)) for i in range(8000)]
    x=[np.linalg.lstsq(np.transpose(A[i])@A[i], np.transpose(A[i])@b[i],rcond=None)[0] for i in range(8000)]
    résidus=[np.linalg.norm(A[i]@x[i]-b[i]) for i in range(8000)]
    return label_train[résidus.index(min(résidus))]

In [ ]:
# LISTES A STOCKER

estim_rotation=[estim_R(i) for i in range(2000)]
estim_rotation_overfit=[estim_R_overfit(i) for i in range(2000)]

## 3.4 Scaling

In [36]:
p_s_train=tuples_x*np.array(p_x_train)+tuples_y*np.array(p_y_train)
p_s_test=tuples_x*np.array(p_x_test)+tuples_y*np.array(p_y_test)

In [ ]:
# ajout overfit
p_s_test_overfit=tuples_x*np.array(p_x_test_overfit)+tuples_y*np.array(p_y_test_overfit)

<div id="ch:method_1"></div>

**Fonction de classification pour le scaling:**

In [38]:
def estim_S(j): #estime l'image data_test[j]
    A=[np.hstack((np.reshape(-p_s_train[i],(-1,1)),np.reshape(p_s_test[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_test[j],(-1,1)) for i in range(8000)]
    résidus=[np.linalg.lstsq(A[i], b[i], rcond=None)[1][0] for i in range(8000)] 
    return label_train[résidus.index(min(résidus))]

In [ ]:
# ajout overfit

def estim_S_overfit(j): #estime l'image data_test_overfit[j]
    A=[np.hstack((np.reshape(-p_s_train[i],(-1,1)),np.reshape(p_s_train[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_train[j],(-1,1)) for i in range(8000)]
    x=[np.linalg.lstsq(np.transpose(A[i])@A[i], np.transpose(A[i])@b[i],rcond=None)[0] for i in range(8000)]
    résidus=[np.linalg.norm(A[i]@x[i]-b[i]) for i in range(8000)]
    return label_train[résidus.index(min(résidus))]

In [ ]:
# LISTES A STOCKER

estim_scaling=[estim_S(i) for i in range(2000)]
estim_scaling_overfit=[estim_S_overfit(i) for i in range(2000)]

## 3.5 Transformation parallèle hyperbolique

In [41]:
p_TPH_train=tuples_x*np.array(p_x_train)-tuples_y*np.array(p_y_train)
p_TPH_test=tuples_x*np.array(p_x_test)-tuples_y*np.array(p_y_test)

In [ ]:
# ajout overfit
p_TPH_test_overfit=tuples_x*np.array(p_x_test_overfit)-tuples_y*np.array(p_y_test_overfit)

<div id="ch:method_1"></div>

**Fonction de classification pour la transformation parallèle hyperbolique:**

In [43]:
def estim_TPH(j): #estime l'image data_test[j]
    A=[np.hstack((np.reshape(-p_TPH_train[i],(-1,1)),np.reshape(p_TPH_test[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_test[j],(-1,1)) for i in range(8000)]
    résidus=[np.linalg.lstsq(A[i], b[i], rcond=None)[1][0] for i in range(8000)] 
    return label_train[résidus.index(min(résidus))]

In [ ]:
# ajout overfit

def estim_TPH_overfit(j): #estime l'image data_test_overfit[j]
    A=[np.hstack((np.reshape(-p_TPH_train[i],(-1,1)),np.reshape(p_TPH_train[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_train[j],(-1,1)) for i in range(8000)]
    x=[np.linalg.lstsq(np.transpose(A[i])@A[i], np.transpose(A[i])@b[i],rcond=None)[0] for i in range(8000)]
    résidus=[np.linalg.norm(A[i]@x[i]-b[i]) for i in range(8000)]
    return label_train[résidus.index(min(résidus))]

In [ ]:
# LISTES A STOCKER

estim_TPH=[estim_TPH(i) for i in range(2000)]
estim_TPH_overfit=[estim_TPH_overfit(i) for i in range(2000)]

## 3.6 Transformation diagonale hyperbolique

In [46]:
p_TDH_train=tuples_y*np.array(p_x_train)+tuples_x*np.array(p_y_train)
p_TDH_test=tuples_y*np.array(p_x_test)+tuples_x*np.array(p_y_test)

In [ ]:
# ajout overfit
p_TDH_test_overfit=tuples_y*np.array(p_x_test_overfit)+tuples_x*np.array(p_y_test_overfit)

<div id="ch:method_1"></div>

**Fonction de classification pour la transformation diagonale hyperbolique:**

In [48]:
def estim_TDH(j): #estime l'image data_test[j]
    A=[np.hstack((np.reshape(-p_TDH_train[i],(-1,1)),np.reshape(p_TDH_test[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_test[j],(-1,1)) for i in range(8000)]
    résidus=[np.linalg.lstsq(A[i], b[i], rcond=None)[1][0] for i in range(8000)] 
    return label_train[résidus.index(min(résidus))]

In [ ]:
# ajout overfit

def estim_TDH_overfit(j): #estime l'image data_test_overfit[j]
    A=[np.hstack((np.reshape(-p_TDH_train[i],(-1,1)),np.reshape(p_TDH_train[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_train[j],(-1,1)) for i in range(8000)]
    x=[np.linalg.lstsq(np.transpose(A[i])@A[i], np.transpose(A[i])@b[i],rcond=None)[0] for i in range(8000)]
    résidus=[np.linalg.norm(A[i]@x[i]-b[i]) for i in range(8000)]
    return label_train[résidus.index(min(résidus))]

In [ ]:
# LISTES A STOCKER

estim_TDH=[estim_TDH(i) for i in range(2000)]
estim_TDH_overfit=[estim_TDH_overfit(i) for i in range(2000)]

## 3.7 Thickening

In [51]:
p_T_train=np.array(p_x_train)**2+np.array(p_y_train)**2
p_T_test=np.array(p_x_test)**2+np.array(p_y_test)**2

In [ ]:
# ajout overfit
p_T_test_overfit=np.array(p_x_test_overfit)**2+np.array(p_y_test_overfit)**2

<div id="ch:method_1"></div>

**Fonction de classification pour le thickening:**

In [53]:
def estim_T(j): #estime l'image data_test[j]
    A=[np.hstack((np.reshape(-p_T_train[i],(-1,1)),np.reshape(p_T_test[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_test[j],(-1,1)) for i in range(8000)]
    résidus=[np.linalg.lstsq(A[i], b[i], rcond=None)[1][0] for i in range(8000)] 
    return label_train[résidus.index(min(résidus))]

In [ ]:
# ajout overfit

def estim_T_overfit(j): #estime l'image data_test_overfit[j]
    A=[np.hstack((np.reshape(-p_T_train[i],(-1,1)),np.reshape(p_T_train[j],(-1,1)))) for i in range(8000)]
    b=[np.reshape(smooth_train[i]-smooth_train[j],(-1,1)) for i in range(8000)]
    x=[np.linalg.lstsq(np.transpose(A[i])@A[i], np.transpose(A[i])@b[i],rcond=None)[0] for i in range(8000)]
    résidus=[np.linalg.norm(A[i]@x[i]-b[i]) for i in range(8000)]
    return label_train[résidus.index(min(résidus))]

In [ ]:
# LISTES A STOCKER

estim_thick=[estim_T(i) for i in range(2000)]
estim_thick_overfit=[estim_T_overfit(i) for i in range(2000)]